<a href="https://colab.research.google.com/github/fininsight/text-mining-tutorial/blob/master/3_%ED%95%B5%EC%8B%AC_%ED%82%A4%EC%9B%8C%EB%93%9C_%EC%B6%94%EC%B6%9C_Keyword_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 핵심 키워드 추출 (Keyword Extraction)

# 1 TF-IDF

# 2 Textrank

## Textrank

In [0]:
df1 = pd.read_csv('preprocessing_실리콘벨리기사.csv')

df1.rename(columns={'0':'raw','1':'tuned'},inplace=True)

doc = df1['raw'].tolist()
sentences = doc[0].split(".")

In [0]:
import networkx as nx
import numpy as np
import pandas as pd
from konlpy.tag import Okt
okt = Okt()

from sklearn.feature_extraction.text import TfidfTransformer

In [0]:
def textrank(document):
    sentences = document[0].split('.')
    tuned_sentences = []

    for idx in range(len(sentences)) : 
        tuned_sentences.append(okt.nouns(sentences[idx]))

    tuned_sentences = [','.join(tuned_sentences[idx]).replace(',',' ') for idx in range(len(tuned_sentences))]
    vectorizer = TfidfVectorizer()
    tfidf_vector = vectorizer.fit_transform(tuned_sentences).toarray()
    similarity_graph = tfidf_vector * tfidf_vector
 
    nx_graph = nx.from_scipy_sparse_matrix(similarity_graph)

    scores = nx.pagerank(nx_graph)
    return sorted(((scores[i],s) for i,s in enumerate(sentences)),
                  reverse=True)

In [0]:
def textrank(document):
    sentences = document.split('.') # 문서를 문장으로 split
    tuned_sentences = []
    
    for idx in range(len(sentences)) : 
        tuned_sentences.append(okt.nouns(sentences[idx]))

    tuned_sentences = [','.join(tuned_sentences[idx]).replace(',',' ') for idx in range(len(tuned_sentences))]
    
    # 위에까지 문장에서 명사만 추출한 후, 명사의 나열로 각각의 문장을 대체한다.
    
    convert_to_binary_ls = CountVectorizer().fit_transform(tuned_sentences)
    tfidf_vetorized_ls = TfidfTransformer().fit_transform(convert_to_binary_ls)
    
    # 위의 두 줄의 코드 ( countervectorizer -> tf-idf transformer) 는 tf-idf vectorizer 과 같은 효과를 가진다.

    similarity_graph = tfidf_vetorized_ls * tfidf_vetorized_ls.T # 벡터의 내적을 통해 코사인 유사도 계산
 
    nx_graph = nx.from_scipy_sparse_matrix(similarity_graph) # 그래프 위에 벡터를 올린다.

    scores = nx.pagerank(nx_graph) # 페이지 랭크 알고리즘으로 노드의 가중치를 업데이트 시켜준다.
    
    return sorted(((scores[i],s) for i,s in enumerate(sentences)),
                  reverse=True)

In [0]:
textrank(doc[0])

[(0.040816762831927554, ' 창업자들의 세부 전공을 보면 37%만 공학·컴퓨터 기술이며, 수학 전공자는 2%뿐이었다'),
 (0.03487320736779421,
  ' 요즘 실리콘밸리에서 확인되는 것은 4차 산업혁명 시대에는 예술과 인문학이 의학·공학만큼 중요하다는 사실이다'),
 (0.03339024696666714, ' 기술 관련 제품의 성공에 가장 중요한 것은 디자인"이라고 답했다'),
 (0.03282283236060363,
  ' 자녀의 직업적 성공을 위해 대학 전공으로 의학과 이공계를 우선적으로 고려하는 일은 한국이 산업화 중이던 상황에선 올바른 선택이었다'),
 (0.03272401428499746,
  '기술을 통해 놀라운 미래를 창조하고 새로운 산업혁명 시대에 리더가 되기 위해 한국에서는 공학자들과 손잡고 일할 음악가와 화가가 더 많이 필요하다'),
 (0.032391493195152554,
  ' 여기에는 컴퓨터와 인공지능, 디지털 의술, 로봇공학, 합성생물학 등 광범위한 기술이 활용된다'),
 (0.032023279817589984,
  '교육을 삶의 최우선 순위로 두고 있는 한국의 부모들은 대학 전공 가운데 의학과 공학·과학을 중시한다'),
 (0.03060327736966526, ' 이는 인문학 전공자들이 가장 잘 훈련받은 분야이다'),
 (0.02977677825410388,
  ' 교양과 인문학이 결합한 기술이야말로 가슴 벅찬 결과를 낳을 것"이라며 예술과 디자인의 중요성을 강조했다'),
 (0.029744919107691305,
  " 초소형 물질의 발전은 모든 사람이 충분히 이용할 수 있는 '태양열 저장 기술'의 새 시대를 열 것이다"),
 (0.02924623343757161,
  ' 최근 내가 그에게 "어떻게 5년여 만에 회사 주가를 450% 올릴 수 있었느냐"고 물었더니 그는 "우리 회사가 만드는 모든 제품의 디자인을 쉼 없이 고민했기 때문에 가능했다'),
 (0.028901992422112152, '실리

In [0]:
from gensim.summarization import summarize
summarize(doc[0])

'요즘 실리콘밸리에서 확인되는 것은 4차 산업혁명 시대에는 예술과 인문학이 의학·공학만큼 중요하다는 사실이다.스티브 잡스는 자신이 대학 시절 수강했던 서체(書體) 수업이 매킨토시(애플이 1984년 발표한 개인용 컴퓨터) 개발 성공에 큰 영향을 미쳤다고 말했다.\n이런 관점을 바탕으로 잡스는 세계 최고 가치를 인정받는 기업을 만들었고, 기술 산업의 새로운 표준까지 정했다.실리콘밸리에서 최근 뜨고 있는 스타 기업인 중에는 인문학 전공자들이 제법 많다.\n구인·구직 소셜 네트워킹 서비스 기업인 링크드인(LinkedIn) 창업자 리드 호프만은 철학 석사 학위 소지자이며, 수잔 보이치키 유튜브 CEO는 역사와 문학을 전공했다.\n메신저 개발 업체 슬랙(Slack)의 창업자 스튜어트 버터필드는 철학, 세계 최대 숙박 공유 기업인 에어비앤비의 설립자 브라이언 체스키는 미술을 전공했다.\n이들의 전공은 경영·회계·보건·예술·인문학 등 매우 다양했다.컴퓨터 주변기기 제조 업체 로지텍의 브랙큰 대럴 CEO도 영문학을 전공했다.'

In [0]:
import pandas as pd
from konlpy.tag import Okt
okt = Okt()

from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
df1 = pd.read_csv('preprocessing_실리콘벨리기사.csv')

In [0]:
df1.rename(columns={'0':'raw','1':'tuned'},inplace=True)

In [0]:
print(df1['raw'].tolist())

['교육을 삶의 최우선 순위로 두고 있는 한국의 부모들은 대학 전공 가운데 의학과 공학·과학을 중시한다. 자녀의 직업적 성공을 위해 대학 전공으로 의학과 이공계를 우선적으로 고려하는 일은 한국이 산업화 중이던 상황에선 올바른 선택이었다. 하지만 지금은 모든 것이 달라졌다. 요즘 실리콘밸리에서 확인되는 것은 4차 산업혁명 시대에는 예술과 인문학이 의학·공학만큼 중요하다는 사실이다.스티브 잡스는 자신이 대학 시절 수강했던 서체(書體) 수업이 매킨토시(애플이 1984년 발표한 개인용 컴퓨터) 개발 성공에 큰 영향을 미쳤다고 말했다. 그는 2011년 아이패드 2를 공개하면서 "애플의 DNA는 기술만으로는 충분하지 않다. 교양과 인문학이 결합한 기술이야말로 가슴 벅찬 결과를 낳을 것"이라며 예술과 디자인의 중요성을 강조했다. 이런 관점을 바탕으로 잡스는 세계 최고 가치를 인정받는 기업을 만들었고, 기술 산업의 새로운 표준까지 정했다.실리콘밸리에서 최근 뜨고 있는 스타 기업인 중에는 인문학 전공자들이 제법 많다. 구인·구직 소셜 네트워킹 서비스 기업인 링크드인(LinkedIn) 창업자 리드 호프만은 철학 석사 학위 소지자이며, 수잔 보이치키 유튜브 CEO는 역사와 문학을 전공했다. 메신저 개발 업체 슬랙(Slack)의 창업자 스튜어트 버터필드는 철학, 세계 최대 숙박 공유 기업인 에어비앤비의 설립자 브라이언 체스키는 미술을 전공했다. 중국 알리바바그룹의 마윈 회장의 학부 전공은 영어였다.내가 속해 있는 하버드대·듀크대 연구팀은 미국 IT 기업 창업자들의 92%가 학사 학위를, 47%는 석사 학위 이상을 갖고 있음을 밝혀냈다. 창업자들의 세부 전공을 보면 37%만 공학·컴퓨터 기술이며, 수학 전공자는 2%뿐이었다. 이들의 전공은 경영·회계·보건·예술·인문학 등 매우 다양했다.컴퓨터 주변기기 제조 업체 로지텍의 브랙큰 대럴 CEO도 영문학을 전공했다. 최근 내가 그에게 "어떻게 5년여 만에 회사 주가를 450% 올릴 수 있었느냐"고 물었더니 그는 "우리 회사가 만드는 모든 

In [0]:
doc = df1['raw'].tolist()
sentences = doc[0].split(".")
sentences

['교육을 삶의 최우선 순위로 두고 있는 한국의 부모들은 대학 전공 가운데 의학과 공학·과학을 중시한다',
 ' 자녀의 직업적 성공을 위해 대학 전공으로 의학과 이공계를 우선적으로 고려하는 일은 한국이 산업화 중이던 상황에선 올바른 선택이었다',
 ' 하지만 지금은 모든 것이 달라졌다',
 ' 요즘 실리콘밸리에서 확인되는 것은 4차 산업혁명 시대에는 예술과 인문학이 의학·공학만큼 중요하다는 사실이다',
 '스티브 잡스는 자신이 대학 시절 수강했던 서체(書體) 수업이 매킨토시(애플이 1984년 발표한 개인용 컴퓨터) 개발 성공에 큰 영향을 미쳤다고 말했다',
 ' 그는 2011년 아이패드 2를 공개하면서 "애플의 DNA는 기술만으로는 충분하지 않다',
 ' 교양과 인문학이 결합한 기술이야말로 가슴 벅찬 결과를 낳을 것"이라며 예술과 디자인의 중요성을 강조했다',
 ' 이런 관점을 바탕으로 잡스는 세계 최고 가치를 인정받는 기업을 만들었고, 기술 산업의 새로운 표준까지 정했다',
 '실리콘밸리에서 최근 뜨고 있는 스타 기업인 중에는 인문학 전공자들이 제법 많다',
 ' 구인·구직 소셜 네트워킹 서비스 기업인 링크드인(LinkedIn) 창업자 리드 호프만은 철학 석사 학위 소지자이며, 수잔 보이치키 유튜브 CEO는 역사와 문학을 전공했다',
 ' 메신저 개발 업체 슬랙(Slack)의 창업자 스튜어트 버터필드는 철학, 세계 최대 숙박 공유 기업인 에어비앤비의 설립자 브라이언 체스키는 미술을 전공했다',
 ' 중국 알리바바그룹의 마윈 회장의 학부 전공은 영어였다',
 '내가 속해 있는 하버드대·듀크대 연구팀은 미국 IT 기업 창업자들의 92%가 학사 학위를, 47%는 석사 학위 이상을 갖고 있음을 밝혀냈다',
 ' 창업자들의 세부 전공을 보면 37%만 공학·컴퓨터 기술이며, 수학 전공자는 2%뿐이었다',
 ' 이들의 전공은 경영·회계·보건·예술·인문학 등 매우 다양했다',
 '컴퓨터 주변기기 제조 업체 로지텍의 브랙큰 대럴 CEO도 영문학을 전공했다',
 ' 최근 내가

In [0]:
tuned_sentences = []

for idx in range(len(sentences)) : 
    tuned_sentences.append(okt.nouns(sentences[idx]))
tuned_sentences[0]

['교육',
 '삶',
 '최',
 '우선',
 '순위',
 '한국',
 '부모',
 '대학',
 '전공',
 '가운데',
 '의학과',
 '공학',
 '과학',
 '중시']

In [0]:
tuned_sentences = [','.join(tuned_sentences[idx]).replace(',',' ') for idx in range(len(tuned_sentences))]
tuned_sentences

['교육 삶 최 우선 순위 한국 부모 대학 전공 가운데 의학과 공학 과학 중시',
 '자녀 직업 성공 위해 대학 전공 의학과 이공 우선 고려 일 한국 산업화 중 상황 선택',
 '지금 모든 것',
 '요즘 실리콘밸리 확인 것 차 산업혁명 시대 예술 인문학 의학 공학 사실',
 '스티브 잡스 자신 대학 시절 수강 서체 수업 매킨토시 애플 발표 개인 용 컴퓨터 개발 성공 영향 말',
 '그 아이패드 를 공개 애플 기술',
 '교양 인문학 결합 기술 가슴 결과 낳을 것 라며 예술 디자인 중요성 강조',
 '관점 바탕 잡스 세계 최고 가치 기업 기술 산업 표준 정',
 '실리콘밸리 최근 스타 기업인 중 인문학 전공자 제법',
 '구인 구직 소셜 네트워킹 서비스 기업인 링크드인 창업 리드 호프 철학 석사 학위 소지 잔 보이 유튜브 역사 문학 전공',
 '메신저 개발 업체 슬랙 의 창업 스튜어트 버터 필드 철학 세계 최대 숙박 공유 기업인 에어비앤비 설립 브라이언 체 스키 미술 전공',
 '중국 알리바바 그룹 마윈 회장 학부 전공 영어',
 '내 해 하버드대 듀크 팀 미국 기업 창업 자 학사 학위 석사 학위 이상 음',
 '창업 자 세부 전공 만 공학 컴퓨터 기술 수학 전공자',
 '이 전공 경영 회계 보건 예술 인문학 등 매우',
 '컴퓨터 주변기기 제조 업체 로지텍 브랙큰 대럴 도 영문학 전공',
 '최근 내 그 회사 주가 수 고 그 우리 회사 모든 제품 디자인 고민 때문',
 '기술 관련 제품 성공 가장 것 디자인 답',
 '차 산업혁명 혁신 규칙 일이',
 '여기 컴퓨터 인공 지능 디지털 의술 로봇공학 합성생물학 등 기술 활용',
 '의학 인공 지능 센서 융합 인간 건강 진단 질병 예방 디지털 의사 를 수',
 '유전체학 유전자 편집 이용 가뭄 인류 전체 새 식물 개발 수',
 '인공 지능 로봇 사용 노인 위 디지털 친구 도',
 '초소 물질 발전 모든 사람 이용 수 태양열 저장 기술 의 새 시대 열 것',
 '해결 책 데 생물학 교육 의료 인간 행동 등 여러 분

In [0]:
vectorizer = TfidfVectorizer()
tfidf_vector = vectorizer.fit_transform(tuned_sentences).toarray()

In [0]:
luhn_importance = []

for idx in range(len(tfidf_vector)) : 
    luhn_importance.append(sum(tfidf_vector[idx]) / len([i for i in tfidf_vector[idx] if i != 0]))
luhn_importance[:5]

/home/hskimim/anaconda3/envs/dss_env/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.


[0.2864330566455741,
 0.2655826626486411,
 0.703848815311294,
 0.3138733801351376,
 0.24904542659640516]

In [0]:
final_df = pd.DataFrame(columns=['sentence','luhn_importance'])
final_df['sentence'] = sentences
final_df['luhn_importance'] = luhn_importance
final_df.sort_values('luhn_importance',ascending=False).head()

,sentence,luhn_importance
2,하지만 지금은 모든 것이 달라졌다,0.703849
18,4차 산업혁명은 '혁신의 규칙'을 바꾸는 일이다,0.498424
5,"그는 2011년 아이패드 2를 공개하면서 ""애플의 DNA는 기술만으로는 충분하지 않다",0.489840
33,그 대신 자녀가 자신의 열정을 추구하고 배우는 걸 즐길 수 있도록 북돋아야 한다,0.445919
26,이는 인문학 전공자들이 가장 잘 훈련받은 분야이다,0.444194
